# Neo4j와 LangChain을 활용한 지식그래프 입문

---

## 1. Neo4J AuraDB 환경 설정

In [2]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [3]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

In [4]:
# 테스트 쿼리 실행 
cypher_query = """
CREATE (n:Test {name: "Hello AuraDB"}) 
RETURN n
"""

graph.query(cypher_query)

[{'n': {'name': 'Hello AuraDB'}}]

In [21]:
def reset_database(graph):
    """
    데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")
    
    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

# 데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


---

## 2. Cypher 쿼리 고급 패턴 매칭

Cypher는 Neo4j의 그래프 쿼리 언어로, 그래프 패턴을 시각적으로 표현하는 구문을 제공합니다.

#### 1. 데이터 모델 구조

소셜 네트워크 데이터 모델을 사용하여 실습

- 노드 레이블:
  - `Person`: 사용자 정보 (이름, 나이, 도시)
  - `Interest`: 관심사 (영화, 음악, 스포츠 등)
  - `Location`: 장소 (남산타워, 해운대 등)

- 관계 유형:
  - `KNOWS`: Person-Person 간의 친구 관계
  - `LIKES`: Person-Interest 간의 관심사 관계
  - `VISITED`: Person-Location 간의 방문 관계


In [22]:
# 데이터 모델 정의 
cypher_query =  """
// Person 노드 생성
CREATE (p1:Person {name: '홍길동', age: 35, city: '서울'})
CREATE (p2:Person {name: '김철수', age: 28, city: '부산'})
CREATE (p3:Person {name: '이영희', age: 32, city: '서울'})
CREATE (p4:Person {name: '박지성', age: 40, city: '인천'})
CREATE (p5:Person {name: '최민수', age: 45, city: '대전'})
CREATE (p6:Person {name: '정소라', age: 29, city: '부산'})
CREATE (p7:Person {name: '강대호', age: 33, city: '서울'})
CREATE (p8:Person {name: '윤미래', age: 27, city: '대구'})

// 관심분야 노드 생성
CREATE (i1:Interest {name: '영화'})
CREATE (i2:Interest {name: '음악'})
CREATE (i3:Interest {name: '스포츠'})
CREATE (i4:Interest {name: '요리'})
CREATE (i5:Interest {name: '여행'})

// 장소 노드 생성
CREATE (l1:Location {name: '남산타워', city: '서울'})
CREATE (l2:Location {name: '해운대', city: '부산'})
CREATE (l3:Location {name: '경복궁', city: '서울'})
CREATE (l4:Location {name: '월미도', city: '인천'})

// KNOWS 관계 생성 (사람들 간의 친구 관계)
CREATE (p1)-[:KNOWS {since: 2010}]->(p2)
CREATE (p1)-[:KNOWS {since: 2015}]->(p3)
CREATE (p1)-[:KNOWS {since: 2012}]->(p7)
CREATE (p2)-[:KNOWS {since: 2018}]->(p6)
CREATE (p3)-[:KNOWS {since: 2017}]->(p4)
CREATE (p3)-[:KNOWS {since: 2020}]->(p5)
CREATE (p4)-[:KNOWS {since: 2019}]->(p5)
CREATE (p6)-[:KNOWS {since: 2016}]->(p8)
CREATE (p7)-[:KNOWS {since: 2014}]->(p3)
CREATE (p7)-[:KNOWS {since: 2021}]->(p8)

// LIKES 관계 생성 (사람과 관심분야 간의 관계)
CREATE (p1)-[:LIKES {rating: 5}]->(i1)
CREATE (p1)-[:LIKES {rating: 4}]->(i5)
CREATE (p2)-[:LIKES {rating: 5}]->(i3)
CREATE (p3)-[:LIKES {rating: 4}]->(i2)
CREATE (p3)-[:LIKES {rating: 5}]->(i4)
CREATE (p4)-[:LIKES {rating: 5}]->(i3)
CREATE (p5)-[:LIKES {rating: 3}]->(i1)
CREATE (p5)-[:LIKES {rating: 4}]->(i5)
CREATE (p6)-[:LIKES {rating: 5}]->(i2)
CREATE (p7)-[:LIKES {rating: 4}]->(i4)
CREATE (p8)-[:LIKES {rating: 4}]->(i5)

// VISITED 관계 생성 (사람과 장소 간의 관계)
CREATE (p1)-[:VISITED {date: '2022-03-15', rating: 4}]->(l1)
CREATE (p1)-[:VISITED {date: '2022-05-20', rating: 5}]->(l2)
CREATE (p2)-[:VISITED {date: '2022-04-10', rating: 3}]->(l2)
CREATE (p3)-[:VISITED {date: '2022-06-05', rating: 5}]->(l3)
CREATE (p4)-[:VISITED {date: '2022-07-12', rating: 4}]->(l4)
CREATE (p4)-[:VISITED {date: '2022-02-28', rating: 5}]->(l1)
CREATE (p5)-[:VISITED {date: '2022-08-03', rating: 4}]->(l3)
CREATE (p6)-[:VISITED {date: '2022-01-17', rating: 5}]->(l2)
CREATE (p7)-[:VISITED {date: '2022-09-22', rating: 3}]->(l1)
CREATE (p8)-[:VISITED {date: '2022-10-08', rating: 4}]->(l2)
"""

graph.query(cypher_query)

[]

#### 2. 가변 길이 경로(Variable-Length Paths)

- 가변 길이 경로는 관계의 길이를 가변적으로 지정할 수 있는 기능입니다. 
- `*` 연산자를 사용하여 표현합니다.


- **예제 1**: 1~3단계 친구 관계 조회

    - 이 쿼리는 홍길동과 1~3단계 떨어진 모든 사람을 찾아 반환합니다. 
    - 결과는 홍길동의 직접적인 친구(김철수, 이영희, 강대호)뿐만 아니라 친구의 친구, 친구의 친구의 친구까지 포함합니다.

In [7]:
cypher_query = """
// 홍길동으로부터 1~3단계 KNOWS 관계로 연결된 모든 사람을 찾는 쿼리입니다
// *1..3 표기법은 1단계부터 3단계까지의 관계를 의미합니다
// - 1단계: 홍길동의 직접적인 친구들
// - 2단계: 홍길동의 친구의 친구들
// - 3단계: 홍길동의 친구의 친구의 친구들
MATCH (p1:Person {name: '홍길동'})-[r:KNOWS*1..3]->(p2:Person)

// 결과를 다음과 같이 반환합니다:
// - 시작점: 홍길동의 이름
// - 관계: 경로상의 각 KNOWS 관계에 대해 관계 타입과 since 속성을 결합한 배열
//   (예: ['KNOWS(2012)', 'KNOWS(2014)'])
// - 도착점: 경로의 끝에 있는 사람의 이름
RETURN p1.name AS 시작점, [rel IN r | type(rel) + '(' + rel.since + ')'] AS 관계, p2.name AS 도착점
"""

graph.query(cypher_query)

[{'시작점': '홍길동', '관계': ['KNOWS(2012)'], '도착점': '강대호'},
 {'시작점': '홍길동', '관계': ['KNOWS(2012)', 'KNOWS(2021)'], '도착점': '윤미래'},
 {'시작점': '홍길동', '관계': ['KNOWS(2012)', 'KNOWS(2014)'], '도착점': '이영희'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2012)', 'KNOWS(2014)', 'KNOWS(2020)'],
  '도착점': '최민수'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2012)', 'KNOWS(2014)', 'KNOWS(2017)'],
  '도착점': '박지성'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)'], '도착점': '이영희'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)', 'KNOWS(2020)'], '도착점': '최민수'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)', 'KNOWS(2017)'], '도착점': '박지성'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2015)', 'KNOWS(2017)', 'KNOWS(2019)'],
  '도착점': '최민수'},
 {'시작점': '홍길동', '관계': ['KNOWS(2010)'], '도착점': '김철수'},
 {'시작점': '홍길동', '관계': ['KNOWS(2010)', 'KNOWS(2018)'], '도착점': '정소라'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2010)', 'KNOWS(2018)', 'KNOWS(2016)'],
  '도착점': '윤미래'}]

- **예제 2**: 정확히 2단계 친구 관계 조회

    - 이 쿼리는 홍길동의 친구의 친구만 반환합니다.

In [10]:
cypher_query = """
// 홍길동으로부터 정확히 2단계 KNOWS 관계로 연결된 사람들만 조회 (친구의 친구)
// *2 표기법은 정확히 2단계의 관계만 찾습니다 (1단계나 3단계는 포함하지 않음)
// 예: 홍길동 -> 이영희 -> 최민수 (홍길동의 친구의 친구)
MATCH (p1:Person {name: '홍길동'})-[:KNOWS*2]->(p2:Person)

// 결과를 다음과 같이 반환합니다:
// - 시작점: 홍길동의 이름
// - 도착점: 2단계 떨어진 사람의 이름 (친구의 친구)
// 이 결과는 홍길동과 직접 연결된 사람은 포함하지 않고, 오직 2단계 경로로만 연결된 사람들만 보여줍니다
RETURN p1.name AS 시작점, p2.name AS 도착점
"""

graph.query(cypher_query)

[{'시작점': '홍길동', '도착점': '이영희'},
 {'시작점': '홍길동', '도착점': '박지성'},
 {'시작점': '홍길동', '도착점': '최민수'},
 {'시작점': '홍길동', '도착점': '정소라'},
 {'시작점': '홍길동', '도착점': '윤미래'}]

- **예제 3**: 무제한 단계 경로 조회

    - 이 쿼리는 홍길동으로부터 시작해서 KNOWS 관계로 연결된 모든 사람을 반환합니다. 
    - `*`는 단계 제한 없이 모든 경로를 탐색합니다.


In [20]:
cypher_query = """
// 홍길동으로부터 시작해 모든 KNOWS 관계를 따라 연결된 사람들 조회
// [:KNOWS*] 표기법은 KNOWS 관계를 통해 연결된 모든 깊이의 경로를 탐색합니다
// 단계 제한이 없으므로 1단계(직접 친구), 2단계(친구의 친구), 3단계 이상 모두 포함됩니다
// 방향성이 있는 -> 연산자를 사용하여 홍길동에서 출발하는 단방향 관계만 조회합니다
MATCH (p1:Person {name: '홍길동'})-[r:KNOWS*]->(p2:Person)

// 결과를 다음과 같이 반환합니다:
// - 시작점: 홍길동의 이름
// - 도착점: KNOWS 관계로 연결된 모든 사람의 이름
// 같은 사람이 여러 경로로 연결되어 있다면 결과에 중복으로 나타날 수 있습니다
RETURN p1.name AS 시작점, p2.name AS 도착점, size(r) AS 관계의_수
"""

graph.query(cypher_query)

[{'시작점': '홍길동', '도착점': '강대호', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '윤미래', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '이영희', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '최민수', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '박지성', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '최민수', '관계의_수': 4},
 {'시작점': '홍길동', '도착점': '이영희', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '최민수', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '박지성', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '최민수', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '김철수', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '정소라', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '윤미래', '관계의_수': 3}]

#### 3. 최단 경로(Shortest Path)

- 두 노드 간의 최단 경로를 찾는 함수입니다. 
- `shortestPath()` 함수를 사용합니다.

- **예제 1**: 두 사람 간의 최단 경로 찾기

    - 이 쿼리는 홍길동에서 최민수까지 갈 수 있는 가장 짧은 경로를 찾아 반환합니다. 
    - 결과는 경로 상의 모든 노드 이름이 배열로 나타납니다.

In [ ]:
cypher_query = """
// 홍길동과 최민수 사이의 최단 경로 조회
// shortestPath() 함수를 사용하여 두 노드 간의 가장 짧은 경로를 찾습니다
// p에 저장되는 값은 홍길동과 최민수 사이의 최단 경로 전체입니다 (path)
// path의 형태는 [시작노드 -> 관계 -> 중간노드 -> 관계 -> 마지막노드] 형식이다.
MATCH p=shortestPath((p1:Person {name: '홍길동'})-[*]-(p2:Person {name: '최민수'}))

// [*] 표기법은 모든 유형의 관계를 고려하며, 방향성 없이 양방향으로 탐색합니다
// 이는 KNOWS, INTERESTED_IN 등 모든 관계 타입을 포함합니다

// nodes(p)는 경로 p에 포함된 모든 노드를 배열로 반환합니다. (즉 관계는 제외하고 노드만 추출)
// nodes 함수는 shortestPath() 함수나 일반적인 MATCH 패턴 탐색을 통해 찾은 **전체 경로(Path)**에서 **노드(Node)**만 걸러낼 때 사용합니다
// [node IN nodes(p) | node.name]은 각 노드의 name 속성만 추출하여 새 배열을 생성합니다
// 결과적으로 '홍길동' -> 'something' -> '최민수'와 같은 경로가 반환됩니다
RETURN [node IN nodes(p) | node.name] AS 경로
"""

graph.query(cypher_query)

[{'경로': ['홍길동', '이영희', '최민수']}]

- **예제 2**: 특정 관계로만 연결된 최단 경로

    - 이 쿼리는 KNOWS 관계만을 사용하여 홍길동에서 최민수까지의 최단 경로를 찾습니다.

In [26]:
cypher_query = """
// KNOWS 관계만 사용하여 홍길동과 최민수 사이의 최단 경로 조회
// shortestPath() 함수를 사용하여 두 사람 간의 가장 짧은 경로를 찾습니다
MATCH p=shortestPath((p1:Person {name: '홍길동'})-[:KNOWS*]-(p2:Person {name: '최민수'}))

// [:KNOWS*] 표기법은 KNOWS 관계만 고려하여 경로를 찾습니다
// 이는 INTERESTED_IN 등 다른 관계 타입은 무시하고 사람 간의 직접적인 관계만 탐색합니다

// 결과는 경로 상의 모든 노드의 이름을 배열로 반환합니다
// 예: ['홍길동', '이영희', '최민수']와 같은 형태로 사람들의 연결 경로가 표시됩니다
// 이를 통해 사회적 네트워크에서 두 사람이 어떻게 연결되어 있는지 확인할 수 있습니다
RETURN [node IN nodes(p) | node.name] AS 경로
"""

graph.query(cypher_query)

[{'경로': ['홍길동', '이영희', '최민수']}]

- **예제 3**: 경로 길이 제한이 있는 최단 경로

    - 이 쿼리는 최대 3단계의 관계 안에서 홍길동에서 정소라까지의 최단 경로를 찾습니다.

In [ ]:
cypher_query = """
// 최대 3단계 관계 내에서 홍길동과 정소라 사이의 최단 경로 조회
// shortestPath() 함수를 사용하여 두 사람 간의 가장 짧은 경로를 찾습니다
MATCH p=shortestPath((p1:Person {name: '홍길동'})-[*..3]-(p2:Person {name: '정소라'}))

// [*..3] 표기법은 최대 3단계까지의 관계만 고려한다는 의미입니다
// 즉, 홍길동에서 정소라까지 최대 3명의 중간 인물을 거쳐 연결되는 경로만 탐색합니다
// 이는 너무 긴 경로를 방지하고 가까운 사회적 연결만 찾고자 할 때 유용합니다
// (개인적인 견해: 근데 어차피 shortestPath 함수는 가장 짧은 경로만 찾기 때문에 이 제한이 크게 의미가 있나 싶기도 함)

// 결과는 경로 상의 모든 노드의 이름을 배열로 반환합니다
// 출력 예시: ['홍길동', '김철수', '정소라']와 같은 형태로 표시됩니다
// 이를 통해 두 사람이 어떤 중간 인물들을 통해 연결되어 있는지 확인할 수 있습니다
RETURN [node IN nodes(p) | node.name] AS 경로
"""

graph.query(cypher_query)

[{'경로': ['홍길동', '김철수', '정소라']}]

#### 4. 집계 함수(Aggregation Functions)

- Cypher는 다양한 집계 함수(COUNT, SUM, AVG, MIN, MAX, COLLECT 등)를 지원합니다.
- 집계함수를 쓰면 집계함수의 대상이 아닌 항목들이 그룹화가 일어난다. 그 후에 집계함수가 동작한다.

- **예제 1**: 사람 수 계산

In [28]:
cypher_query = """
// 전체 Person 노드 수 계산
// MATCH (p:Person)은 그래프 데이터베이스에서 모든 Person 레이블을 가진 노드를 찾습니다
MATCH (p:Person)

// COUNT(p)는 매칭된 Person 노드의 총 개수를 계산하는 집계 함수입니다
// AS 전체사람수는 결과 컬럼의 이름을 '전체사람수'로 지정합니다
// 이 쿼리는 데이터베이스에 저장된 모든 사람(Person) 노드의 총 개수를 반환합니다
// 결과는 단일 숫자 값으로, 네트워크에 포함된 전체 인물의 수를 나타냅니다
RETURN COUNT(p) AS 전체사람수
"""

graph.query(cypher_query)

[{'전체사람수': 8}]

- **예제 2**: 도시별 사람 수 계산

In [ ]:
cypher_query = """
// 도시별 Person 노드 수 계산
// MATCH (p:Person)은 그래프 데이터베이스에서 모든 Person 레이블을 가진 노드를 찾습니다
MATCH (p:Person)

// p.city는 각 Person 노드의 city 속성을 나타냅니다
// COUNT(p)는 각 도시별로 매칭된 Person 노드의 개수를 계산하는 집계 함수입니다
// AS 구문은 결과 컬럼의 이름을 '도시'와 '사람수'로 지정합니다
// ★ 쿼리에서 가장 먼저 MATCH (p:Person)을 수행하는 이유는 '어떤 데이터'를 찾아서 계산할 것인지를 Neo4j 데이터베이스에 지정해주는 가장 기본적인 단계이기 때문이다.
// ★ COUNT(p)는 **집계 함수**이기 때문에, COUNT(p)를 제외한 다른 RETURN 항목(여기서는 p.city)을 기준으로 자동으로 그룹화가 일어난다.
RETURN p.city AS 도시, COUNT(p) AS 사람수

// ORDER BY 사람수 DESC는 결과를 '사람수' 기준으로 내림차순 정렬합니다
// 이를 통해 가장 많은 사람이 사는 도시부터 순서대로 볼 수 있습니다
// 결과는 서울(3명), 부산(2명) 등과 같이 각 도시별 인구 분포를 보여줍니다
ORDER BY 사람수 DESC
"""

graph.query(cypher_query)

[{'p': {'city': '서울', 'name': '홍길동', 'age': 35}},
 {'p': {'city': '부산', 'name': '김철수', 'age': 28}},
 {'p': {'city': '서울', 'name': '이영희', 'age': 32}},
 {'p': {'city': '인천', 'name': '박지성', 'age': 40}},
 {'p': {'city': '대전', 'name': '최민수', 'age': 45}},
 {'p': {'city': '부산', 'name': '정소라', 'age': 29}},
 {'p': {'city': '서울', 'name': '강대호', 'age': 33}},
 {'p': {'city': '대구', 'name': '윤미래', 'age': 27}}]

- **예제 3**: 평균 나이 계산

In [31]:
cypher_query = """
// 전체 사람들의 평균 나이 계산
// MATCH (p:Person)은 그래프 데이터베이스에서 모든 Person 레이블을 가진 노드를 찾습니다
MATCH (p:Person)

// AVG(p.age)는 모든 Person 노드의 age 속성 값의 평균을 계산하는 집계 함수입니다
// AS 평균나이는 결과 컬럼의 이름을 '평균나이'로 지정합니다
// 이 쿼리는 데이터베이스에 저장된 모든 사람(Person)의 나이 평균을 계산합니다
// 결과는 단일 숫자 값으로, 네트워크에 포함된 전체 인물의 평균 연령을 나타냅니다
RETURN AVG(p.age) AS 평균나이
"""

graph.query(cypher_query)

[{'평균나이': 33.62500000000001}]

- **예제 4**: 관심분야별 좋아하는 사람 수

In [36]:
cypher_query = """
// 각 관심분야를 좋아하는 사람 수 계산
// MATCH (p:Person)-[:LIKES]->(i:Interest)는 Person 노드에서 LIKES 관계를 통해 Interest 노드로 연결된 패턴을 찾습니다
// p는 사람 노드, i는 관심분야 노드를 나타냅니다
MATCH (p:Person)-[:LIKES]->(i:Interest)

// i.name AS 관심분야는 각 Interest 노드의 name 속성을 '관심분야'라는 이름으로 반환합니다
// COUNT(p) AS 사람수는 각 관심분야별로 연결된 Person 노드의 수를 계산하여 '사람수'라는 이름으로 반환합니다
// 이를 통해 각 관심분야마다 몇 명의 사람이 좋아하는지 집계할 수 있습니다
RETURN i.name AS 관심분야, COUNT(p) AS 사람수

// ORDER BY 사람수 DESC는 결과를 '사람수' 기준으로 내림차순 정렬합니다
// 이를 통해 가장 많은 사람들이 좋아하는 관심분야부터 순서대로 볼 수 있습니다
// 결과는 여행(3명), 영화(2명), 스포츠(2명) 등과 같이 각 관심분야별 인기도를 보여줍니다
ORDER BY 사람수 DESC
"""

graph.query(cypher_query)

[{'관심분야': '여행', '사람수': 3},
 {'관심분야': '영화', '사람수': 2},
 {'관심분야': '스포츠', '사람수': 2},
 {'관심분야': '음악', '사람수': 2},
 {'관심분야': '요리', '사람수': 2}]

- **예제 5**: 가장 높은 평점의 장소 찾기

In [37]:
cypher_query = """
// 방문한 장소 중 평균 평점이 가장 높은 곳 찾기
// MATCH (p:Person)-[v:VISITED]->(l:Location)은 Person 노드에서 VISITED 관계를 통해 Location 노드로 연결된 패턴을 찾습니다
// p는 사람 노드, v는 방문 관계, l은 장소 노드를 나타냅니다
MATCH (p:Person)-[v:VISITED]->(l:Location)

// l.name AS 장소는 각 Location 노드의 name 속성을 '장소'라는 이름으로 반환합니다
// AVG(v.rating) AS 평균평점은 각 장소별로 모든 방문 관계(v)의 rating 속성 평균을 계산하여 '평균평점'이라는 이름으로 반환합니다
// 이를 통해 각 장소마다 방문자들이 준 평점의 평균값을 구할 수 있습니다
RETURN l.name AS 장소, AVG(v.rating) AS 평균평점

// ORDER BY 평균평점 DESC는 결과를 '평균평점' 기준으로 내림차순 정렬합니다
// 이를 통해 평균 평점이 가장 높은 장소부터 순서대로 볼 수 있습니다
ORDER BY 평균평점 DESC

// LIMIT 1은 결과 중 첫 번째 행만 반환하도록 제한합니다
// 이를 통해 평균 평점이 가장 높은 장소 하나만 결과로 얻을 수 있습니다
// 결과는 경복궁(평균평점 4.5)과 같이 가장 높은 평점을 받은 장소 하나만 표시됩니다
LIMIT 1
"""

graph.query(cypher_query)

[{'장소': '경복궁', '평균평점': 4.5}]

#### 5. WITH 절

- `WITH` 절은 중간 결과를 다음 쿼리 부분에 전달할 수 있게 해줍니다.

- **예제 1**: 기본 필터링

In [41]:
cypher_query = """
// 20세 이상인 사람 중에서 이름에 '김'이 포함된 사람 찾기
// MATCH (p:Person)은 그래프에서 Person 레이블을 가진 모든 노드를 찾아 p 변수에 할당합니다
MATCH (p:Person)

// WHERE p.age >= 20은 p 노드의 age 속성이 20 이상인 노드만 필터링합니다
// 이를 통해 20세 이상인 사람만 선택됩니다
WHERE p.age >= 20

// WITH p는 이전 단계에서 필터링된 결과(p)를 다음 쿼리 부분으로 전달합니다
// WITH 절은 쿼리의 중간 결과를 다음 단계로 파이프라인처럼 전달하는 역할을 합니다
WITH p

// WHERE p.name CONTAINS '김'은 p 노드의 name 속성에 '김'이라는 문자열이 포함된 노드만 추가로 필터링합니다
// 이를 통해 20세 이상이면서 이름에 '김'이 포함된 사람만 선택됩니다
WHERE p.name CONTAINS '김'

// RETURN p.name AS 이름, p.age AS 나이는 최종 결과로 선택된 노드의 name 속성을 '이름'으로, 
// age 속성을 '나이'로 이름을 붙여 반환합니다
// 결과적으로 20세 이상이면서 이름에 '김'이 포함된 사람의 이름과 나이가 출력됩니다
RETURN p.name AS 이름, p.age AS 나이
"""

graph.query(cypher_query)

[{'이름': '김철수', '나이': 28}]

- **예제 2**: 집계 결과를 중간 처리한 후 다시 활용

In [42]:
cypher_query = """
// 각 도시별 평균 나이를 계산한 후, 평균 나이가 35세 이상인 도시만 반환
// MATCH (p:Person)은 그래프에서 Person 레이블을 가진 모든 노드를 찾아 p 변수에 할당합니다
MATCH (p:Person)

// WITH p.city AS 도시, AVG(p.age) AS 평균나이는 다음을 수행합니다:
// 1. p.city 값을 '도시'라는 변수에 그룹화합니다
// 2. 각 도시 그룹별로 p.age의 평균값을 계산하여 '평균나이'라는 변수에 저장합니다
// 3. 이 중간 결과를 다음 쿼리 부분으로 전달합니다
WITH p.city AS 도시, AVG(p.age) AS 평균나이

// WHERE 평균나이 >= 35는 평균 나이가 35세 이상인 도시 그룹만 필터링합니다
// 이를 통해 젊은 인구 분포를 가진 도시는 결과에서 제외됩니다
WHERE 평균나이 >= 35

// RETURN 도시, 평균나이는 필터링된 결과에서 도시명과 해당 도시의 평균 나이를 반환합니다
RETURN 도시, 평균나이

// ORDER BY 평균나이 DESC는 결과를 평균 나이를 기준으로 내림차순 정렬합니다
// 즉, 평균 나이가 가장 높은 도시가 결과의 맨 위에 표시됩니다
ORDER BY 평균나이 DESC
"""

graph.query(cypher_query)

[{'도시': '대전', '평균나이': 45.0}, {'도시': '인천', '평균나이': 40.0}]

- **예제 3**: 복잡한 집계 작업

In [40]:
cypher_query = """
// 사람별로 방문한 장소 수와 관심분야 수를 계산하여, 둘 다 2개 이상인 사람 찾기
// MATCH (p:Person)은 그래프에서 Person 레이블을 가진 모든 노드를 찾아 p 변수에 할당합니다
MATCH (p:Person)

// OPTIONAL MATCH (p)-[:VISITED]->(l:Location)은 다음을 수행합니다:
// 1. 각 사람(p)에서 VISITED 관계를 통해 연결된 Location 노드를 찾습니다
// 2. OPTIONAL MATCH를 사용하여 방문한 장소가 없는 사람도 결과에 포함시킵니다
// 3. 방문 장소가 없는 사람의 경우 l은 NULL이 됩니다
OPTIONAL MATCH (p)-[:VISITED]->(l:Location)

// WITH p, COUNT(DISTINCT l) AS 방문장소수는 다음을 수행합니다:
// 1. 각 사람(p)별로 방문한 고유 장소(l)의 수를 계산합니다
// 2. DISTINCT를 사용하여 중복 방문은 한 번만 계산합니다
// 3. 이 중간 결과를 '방문장소수'라는 변수에 저장하고 다음 쿼리 부분으로 전달합니다
WITH p, COUNT(DISTINCT l) AS 방문장소수

// OPTIONAL MATCH (p)-[:LIKES]->(i:Interest)는 다음을 수행합니다:
// 1. 각 사람(p)에서 LIKES 관계를 통해 연결된 Interest 노드를 찾습니다
// 2. OPTIONAL MATCH를 사용하여 관심사가 없는 사람도 결과에 포함시킵니다
// 3. 관심사가 없는 사람의 경우 i는 NULL이 됩니다
OPTIONAL MATCH (p)-[:LIKES]->(i:Interest)

// WITH p, 방문장소수, COUNT(DISTINCT i) AS 관심분야수는 다음을 수행합니다:
// 1. 이전 단계에서 계산한 '방문장소수'를 유지합니다
// 2. 각 사람별로 관심 있는 고유 분야(i)의 수를 계산합니다
// 3. DISTINCT를 사용하여 같은 관심사는 한 번만 계산합니다
// 4. 이 결과를 '관심분야수'라는 변수에 저장하고 다음 쿼리 부분으로 전달합니다
WITH p, 방문장소수, COUNT(DISTINCT i) AS 관심분야수

// WHERE 방문장소수 >= 2 AND 관심분야수 >= 2는 다음을 수행합니다:
// 1. 방문한 장소가 2곳 이상이고 동시에 관심 분야가 2개 이상인 사람만 필터링합니다
// 2. 이 조건을 충족하지 않는 사람은 결과에서 제외됩니다
WHERE 방문장소수 >= 2 AND 관심분야수 >= 2

// RETURN p.name AS 이름, 방문장소수, 관심분야수는 다음을 수행합니다:
// 1. 필터링된 결과에서 사람의 이름(p.name)을 '이름'으로 표시합니다
// 2. 각 사람이 방문한 장소의 수(방문장소수)를 표시합니다
// 3. 각 사람의 관심 분야 수(관심분야수)를 표시합니다
// 4. 최종적으로 방문 장소와 관심 분야가 모두 2개 이상인 사람의 정보가 출력됩니다
RETURN p.name AS 이름, 방문장소수, 관심분야수
"""

graph.query(cypher_query)

[{'이름': '홍길동', '방문장소수': 2, '관심분야수': 2}]

#### 6. 복잡한 패턴 결합

- 여러 고급 기능을 함께 사용하여 복잡한 쿼리를 작성할 수 있습니다.

- **예제 1**: 같은 관심사를 가진 친구의 친구 찾기

In [43]:
cypher_query = """
// MATCH (p1:Person)-[:LIKES]->(i:Interest {name: '영화'})는 다음을 수행합니다:
// 1. 영화에 관심이 있는 사람(p1)을 찾습니다
// 2. Person 노드에서 LIKES 관계를 통해 Interest 노드로 연결된 패턴을 찾습니다
// 3. Interest 노드 중에서 name 속성이 '영화'인 노드만 필터링합니다
// 4. 이 패턴에 매칭되는 모든 사람(p1)과 영화 관심사(i)를 식별합니다
MATCH (p1:Person)-[:LIKES]->(i:Interest {name: '영화'})

// MATCH (p1)-[:KNOWS*2]->(p2:Person)-[:LIKES]->(i)는 다음을 수행합니다:
// 1. 앞서 찾은 영화 관심사가 있는 사람(p1)에서 시작합니다
// 2. KNOWS 관계를 정확히 2단계 거쳐 연결된 다른 사람(p2)을 찾습니다
//    (예: p1 -> 중간사람 -> p2와 같은 형태로, 친구의 친구를 찾음)
// 3. 그 사람(p2)이 같은 영화 관심사(i)를 가지고 있는지 확인합니다
// 4. p2에서 LIKES 관계를 통해 앞서 찾은 동일한 Interest 노드(i)로 연결되어 있어야 합니다
MATCH (p1)-[:KNOWS*2]->(p2:Person)-[:LIKES]->(i)

// WHERE p1 <> p2는 다음을 수행합니다:
// 1. p1과 p2가 서로 다른 사람인지 확인합니다
// 2. 자기 자신으로 돌아오는 경로는 제외합니다
// 3. 이 조건이 없으면 특정 네트워크 구조에서 시작점과 동일한 사람이 결과에 포함될 수 있습니다
WHERE p1 <> p2

// RETURN p1.name AS 시작사람, p2.name AS 연결된사람은 다음을 수행합니다:
// 1. 조건을 만족하는 두 사람의 이름을 반환합니다
// 2. p1의 이름은 '시작사람'으로, p2의 이름은 '연결된사람'으로 열 이름을 지정합니다
// 3. 최종적으로 영화에 관심이 있는 사람과, 그 사람으로부터 정확히 2단계 떨어져 있으면서
//    역시 영화에 관심이 있는 다른 사람의 쌍을 보여줍니다
RETURN p1.name AS 시작사람, p2.name AS 연결된사람
"""

graph.query(cypher_query)

[{'시작사람': '홍길동', '연결된사람': '최민수'}]

- **예제 2**: 친구 관계와 공통 관심사 결합

In [44]:
cypher_query = """
// MATCH (p1:Person {name: '홍길동'})-[:LIKES]->(i:Interest)<-[:LIKES]-(p2:Person)는 다음을 수행합니다:
// 1. p1은 이름이 '홍길동'인 Person 노드를 찾습니다
// 2. 홍길동(p1)이 LIKES 관계를 통해 연결된 관심사(i) 노드를 찾습니다
// 3. 그 관심사(i)에 LIKES 관계로 연결된 다른 사람들(p2)을 찾습니다
// 4. 이렇게 하면 홍길동과 같은 관심사를 가진 모든 사람들이 식별됩니다

// WHERE NOT (p1)-[:KNOWS]-(p2)는 다음을 수행합니다:
// 1. 홍길동(p1)과 다른 사람(p2) 사이에 직접적인 KNOWS 관계가 없는지 확인합니다
// 2. 즉, 홍길동과 직접 친구 관계가 아닌 사람들만 필터링합니다
// 3. 이 조건으로 홍길동과 이미 알고 있는 사람들은 결과에서 제외됩니다

// RETURN DISTINCT p2.name AS 이름, i.name AS 공통관심사는 다음을 수행합니다:
// 1. 조건을 만족하는 사람의 이름과 홍길동과 공유하는 관심사를 반환합니다
// 2. DISTINCT 키워드는 중복된 결과를 제거합니다
//    (한 사람이 홍길동과 여러 관심사를 공유할 경우 각 관심사별로 별도 행으로 표시됨)
// 3. 최종적으로 홍길동과 직접 친구는 아니지만 같은 관심사를 가진 사람과 그 관심사를 보여줍니다
MATCH (p1:Person {name: '홍길동'})-[:LIKES]->(i:Interest)<-[:LIKES]-(p2:Person)
WHERE NOT (p1)-[:KNOWS]-(p2)
RETURN DISTINCT p2.name AS 이름, i.name AS 공통관심사
"""

graph.query(cypher_query)

[{'이름': '최민수', '공통관심사': '영화'},
 {'이름': '최민수', '공통관심사': '여행'},
 {'이름': '윤미래', '공통관심사': '여행'}]

- **예제 3**: 최단 경로와 집계 함수 결합

In [ ]:
cypher_query = """
// MATCH p=shortestPath((start:Person {name: '홍길동'})-[:KNOWS*]-(other:Person))는 다음을 수행합니다:
// 1. start 변수에 이름이 '홍길동'인 Person 노드를 할당합니다
// 2. other 변수에는 다른 Person 노드들을 할당합니다
// 3. shortestPath 함수를 사용하여 홍길동에서 다른 사람들까지의 최단 경로를 찾습니다
// 4. [:KNOWS*]는 KNOWS 관계가 1번 이상 반복될 수 있음을 의미합니다 (경로의 길이 제한 없음)
// 5. 찾은 최단 경로를 변수 p에 저장합니다

// WHERE start <> other는 다음을 수행합니다:
// 1. start와 other가 서로 다른 노드인지 확인합니다
// 2. 즉, 홍길동 자신은 결과에서 제외합니다

// WITH other, length(p) AS 거리는 다음을 수행합니다:
// 1. other 노드(다른 사람)와 최단 경로의 길이를 다음 단계로 전달합니다
// 2. length(p)는 경로 p의 관계 수를 계산하여 '거리'라는 이름으로 저장합니다
// 3. 이 거리는 홍길동으로부터 other까지 도달하는 데 필요한 KNOWS 관계의 수입니다

// RETURN 거리, COUNT(other) AS 사람수는 다음을 수행합니다:
// 1. 각 거리 값과 해당 거리에 있는 사람 수를 반환합니다
// 2. COUNT(other)는 각 거리에 있는 사람들의 수를 집계합니다

// ORDER BY 거리는 다음을 수행합니다:
// 1. 결과를 거리 값에 따라 오름차순으로 정렬합니다
// 2. 즉, 홍길동으로부터 가까운 사람들부터 먼 사람들 순으로 정렬됩니다
MATCH p=shortestPath((start:Person {name: '홍길동'})-[:KNOWS*]-(other:Person))
WHERE start <> other
WITH other, length(p) AS 거리
RETURN 거리, COUNT(other) AS 사람수
ORDER BY 거리
"""

graph.query(cypher_query)
